(pandas_dataframes)=
# Pandas `DataFrame` Objects

We start with a quote from the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) for the `pd.DataFrame` object:

> Two-dimensional, size-mutable, potentially heterogeneous tabular data. [...] Can be thought of as a dict-like container for Series objects. The primary pandas data structure.

We will be honest with you... `DataFrame`s are *by far* the more interesting and useful than `Series`. "Then why bother with `Series`?" you may ask. Well, we feel that knowing the basics well indeed is worthwhile. `Series` are an integral part of `DataFrame`s (more on this below) and so, sooner or later, we would have dealt with them anyway. Additionally, much of what we have learned about `Series` can be applied --- or transferred with ease --- to "`DataFrame` land". So, let's go right at it ...

To start with, take a look at the following sketch that tries its best to give an overview over the `DataFrame` object. As we can see a key difference between a `Series` and a `DataFrame` is that the latter can have multiple columns. These columns all share the same index and each of them is a fully-fledged `Series`. As a result a `DataFrame` is the go-to tool for handling datasets with multiple features. Usually, each feature will be a column. The columns may have names that describe the feature if possible (for some datasets we might not know what the features actually represent). 

![](../../_build_img/PandasDataFrame-1.png)

Before we dive into the details of `DataFrames` we have to make a few `import`s.

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

f"Pandas version: {pd.__version__ = }, Numpy version: {np.__version__ = }"

"Pandas version: pd.__version__ = '2.2.3', Numpy version: np.__version__ = '2.2.3'"

## `DataFrame` creation

As usual, we start with creating `DataFrames`. Since they are two-dimensional we can initialize them from nested data stuctures such a nested `list` or a `np.ndarray`.

In [2]:
pd.DataFrame([[1, 2, 3], [4, 5, 6]])

,0,1,2
0,1,2,3
1,4,5,6


In [3]:
pd.DataFrame(np.arange(1, 7).reshape((2, 3)))

,0,1,2
0,1,2,3
1,4,5,6


While this for of initialization is straight forward, the result is not very useful. Obviously, we would (at least) like the columns to have some decent names. By default, i.e if no column labels are given, we are provided with integer values --- just as for the index. So, here we go and specify some names for the columns.

In [4]:
pd.DataFrame(np.arange(1, 7).reshape((2, 3)), columns=["a", "b", "c"])

,a,b,c
0,1,2,3
1,4,5,6


And, for the sake of completeness, we initialize the index as well.

In [5]:
pd.DataFrame(
    np.arange(1, 7).reshape((2, 3)), 
    columns=["a", "b", "c"],
    index=["row 1", "row 2"]
)

,a,b,c
row 1,1,2,3
row 2,4,5,6


Other ways to initialize a `DataFrame` are the following:

In [6]:
name = ["person 1", "person 2", "person 3"]
age = [23, 27, 34]

In [7]:
pd.DataFrame(
    data=zip(name, age),
    columns=["Name", "Alter"]
)

,Name,Alter
0,person 1,23
1,person 2,27
2,person 3,34


In [8]:
pd.DataFrame(
    {
        "Name": name,
        "Alter": age,
    }
)

,Name,Alter
0,person 1,23
1,person 2,27
2,person 3,34


From the examples for creating a `DataFrames` you have been able to guess the names of the attributes.[^common-dataframe-variable-names]

[^common-dataframe-variable-names]: While this might be considered an anti pattern by some people, it is common to use variables names like `df` (short for dataframe) or `df_<something descriptive goes here>`.

In [9]:
df = pd.DataFrame({"Name": name, "Age": age})

In [10]:
df.index

RangeIndex(start=0, stop=3, step=1)

In [11]:
df.shape

(3, 2)

In [12]:
df.dtypes

Name    object
Age      int64
dtype: object

So far nothing new. However, we now also have `.dtypes` and  `.columns` attributes where the latter that contains the names of all columns.

In [13]:
df.columns

Index(['Name', 'Age'], dtype='object')

In [14]:
df.to_numpy()

array([['person 1', 23],
       ['person 2', 27],
       ['person 3', 34]], dtype=object)

We can obtain a compact overview of some important information when calling the [`.info()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html) method.

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    3 non-null      object
 1   Age     3 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 180.0+ bytes


Let's quickly revisit the call to the `.to_numpy()`method.

In [16]:
df.to_numpy()

array([['person 1', 23],
       ['person 2', 27],
       ['person 3', 34]], dtype=object)

As you can see we receive a Numpy array with `dtype == object`. The reason for this is that the array must encompass the *data from all columns* of the `DataFrame`. NumPy's real power lies in performant number crunching which requires numerical data types (some integer or some floating point data types). Since columns `DataFrame`s can evidently have different `dtype`s (cf. the content of `.dtypes`) packing all of them in a `ndarray` requires finding something like a "common deminator" for all types. If there is a column with `str` entries the `ndarray` cannot have a numerical type and we get `object`. Operations on arrays with numerical and non-numerical datatypes are not well-defined in all cases (after all, a `+` has a different meaning for `float` than for `str`). Additionally, operations with `dtype == object` arrays are not near as performant as with numerical `dtype`s. In short, calling `.to_values()` only makes sense if all columns have a numerical `dtype`.[^encoding-of-non-numerical-data] Even then, bear in mind that some types will be converted (like `int` to `float`) in the process of creating the `ndarrays`.

[^encoding-of-non-numerical-data]: In many datasets you will have numerical as well as non-numerical features. Machine learning algorithms usually require us to bring all the data into a numerical representation. Therefore, we usually *encode* non-numerical data in a suitable manner (e.g., ordinal encoding for categorical variables).

In [17]:
df = pd.DataFrame({"integers": [1, 2, 3], "floats": [10.0, 20.0, 30.0]})
df

,integers,floats
0,1,10.0
1,2,20.0
2,3,30.0


In [18]:
df.dtypes

integers      int64
floats      float64
dtype: object

All integers will be type-cast to `float64`.

In [19]:
df.to_numpy()

array([[ 1., 10.],
       [ 2., 20.],
       [ 3., 30.]])

### Quiz

<span style="display:none" id="5_DataFrames:1">W3sidHlwZSI6ICJtYW55X2Nob2ljZSIsICJhbnN3ZXJfY29scyI6IDMsICJxdWVzdGlvbiI6ICJXaGljaCBvZiB0aGUgZm9sbG93aW5nIGF0dHJpYnV0ZXMgZG8gUGFuZGFzIGBEYXRhRnJhbWVzYCBoYXZlPyIsICJhbnN3ZXJzIjogW3siY29ycmVjdCI6IGZhbHNlLCAiYW5zd2VyIjogIiAiLCAiY29kZSI6ICIuYXJyYXkifSwgeyJjb3JyZWN0IjogZmFsc2UsICJhbnN3ZXIiOiAiICIsICJjb2RlIjogIi5yb3dzIn0sIHsiY29ycmVjdCI6IHRydWUsICJhbnN3ZXIiOiAiICIsICJjb2RlIjogIi5pbmRleCJ9LCB7ImNvcnJlY3QiOiB0cnVlLCAiYW5zd2VyIjogIiAiLCAiY29kZSI6ICIuY29sdW1ucyJ9LCB7ImNvcnJlY3QiOiB0cnVlLCAiYW5zd2VyIjogIiAiLCAiY29kZSI6ICIudmFsdWVzIn0sIHsiY29ycmVjdCI6IGZhbHNlLCAiYW5zd2VyIjogIiAiLCAiY29kZSI6ICIuZHR5cGUifSwgeyJjb3JyZWN0IjogdHJ1ZSwgImFuc3dlciI6ICIgIiwgImNvZGUiOiAiLnNpemUifSwgeyJjb3JyZWN0IjogdHJ1ZSwgImFuc3dlciI6ICIgIiwgImNvZGUiOiAiLnNoYXBlIn0sIHsiY29ycmVjdCI6IGZhbHNlLCAiYW5zd2VyIjogIiAiLCAiY29kZSI6ICIucm93cyJ9XX1d</span>

In [20]:
import jupyterquiz
jupyterquiz.display_quiz("#5_DataFrames:1")

<IPython.core.display.Javascript object>

<span style="display:none" id="5_DataFrames:2">W3sidHlwZSI6ICJtYW55X2Nob2ljZSIsICJhbnN3ZXJfY29scyI6IDEsICJxdWVzdGlvbiI6ICJZb3UgYXJlIGdpdmVuIHR3byBgbmRhcnJheXNgIChzYW1lIHNpemUpIHdpdGggbmFtZXMgYHYxLCBhbmQgYHYyYC4gV2hpY2ggb2YgdGhlIGZvbGxvd2luZyBleHByZXNzaW9ucyB3aWxsIGNyZWF0ZSBhIGBEYXRhRnJhbWVgPyIsICJhbnN3ZXJzIjogW3siY29ycmVjdCI6IHRydWUsICJhbnN3ZXIiOiAiICIsICJjb2RlIjogInBkLkRhdGFGcmFtZShbdjEsIHYyXSkifSwgeyJjb3JyZWN0IjogdHJ1ZSwgImFuc3dlciI6ICIgIiwgImNvZGUiOiAicGQuRGF0YUZyYW1lKHppcCh2MSwgdjIpKSJ9LCB7ImNvcnJlY3QiOiBmYWxzZSwgImFuc3dlciI6ICIgIiwgImNvZGUiOiAicGQuRGF0YUZyYW1lKHYxLCB2MikifSwgeyJjb3JyZWN0IjogZmFsc2UsICJhbnN3ZXIiOiAiICIsICJjb2RlIjogInBkLkRhdGFGcmFtZSh7djEsIHYyfSkifSwgeyJjb3JyZWN0IjogdHJ1ZSwgImFuc3dlciI6ICIgIiwgImNvZGUiOiAicGQuRGF0YUZyYW1lKHtcImNvbHVtbiAxXCI6IHYxLCBcImNvbHVtbiAyXCI6IHYyfSkifV19XQ==</span>

In [21]:

jupyterquiz.display_quiz("#5_DataFrames:2")

<IPython.core.display.Javascript object>

You are given the following `DataFrame` (as a table).

| Index | A |   B  |  C  |
|:-----:|:-:|:----:|:---:|
|   a   | 1 | 40.0 | "a" |
|   b   | 2 | 30.0 | "b" |
|   c   | 3 | 20.0 | "c" |
|   d   | 4 | 10.0 | "d" |

<span style="display:none" id="5_DataFrames:3">W3sidHlwZSI6ICJtYW55X2Nob2ljZSIsICJhbnN3ZXJfY29scyI6IDEsICJxdWVzdGlvbiI6ICJXaGF0IGlzIHRoZSBgZHR5cGVgIG9idGFpbmVkIHdoZW4gY2FsbGluZyB0aGUgYC50b19udW1weSgpYCBtZXRob2Q/IiwgImFuc3dlcnMiOiBbeyJjb3JyZWN0IjogZmFsc2UsICJhbnN3ZXIiOiAiICIsICJjb2RlIjogImZsb2F0NjQifSwgeyJjb3JyZWN0IjogZmFsc2UsICJhbnN3ZXIiOiAiICIsICJjb2RlIjogImNvbXBsZXgxMjgifSwgeyJjb3JyZWN0IjogZmFsc2UsICJhbnN3ZXIiOiAiICIsICJjb2RlIjogInVpbnQzMiJ9LCB7ImNvcnJlY3QiOiB0cnVlLCAiYW5zd2VyIjogIiAiLCAiY29kZSI6ICJvYmplY3QifSwgeyJjb3JyZWN0IjogZmFsc2UsICJhbnN3ZXIiOiAiICIsICJjb2RlIjogInN0ciJ9XX1d</span>

In [22]:

jupyterquiz.display_quiz("#5_DataFrames:3")

<IPython.core.display.Javascript object>

### Exercises

#### `DataFrame` creation 1

Given the `list` below generate a `DataFrame` in two different ways. The names of the columns shall be `"random numbers"` (for `values1`) and `"countdown"` (for `values2`).

In [23]:
values1 = np.random.randint(-10, 10, size=5)
values2 = range(5, 0, -1)

First solution.

In [24]:
pd.DataFrame(data=zip(values1, values2), columns=["random numbers", "countdown"])

,random numbers,countdown
0,5,5
1,4,4
2,3,3
3,-6,2
4,-10,1


Second solution.

In [25]:
pd.DataFrame({'random numbers': values1, "countdown": values2})

,random numbers,countdown
0,5,5
1,4,4
2,3,3
3,-6,2
4,-10,1


#### `DataFrame` creation 2

Generate a NumPy array with shape `(10, 10)` containing values 1, 2, ..., 100. Use this array to create a `DataFrame` with column names `"col1"`, `"col2"`, ..., `"col10"`, and index `"row1"`, `"row2"`, ..., `"row10"`.

In [26]:
pd.DataFrame(
    data=np.arange(1, 101).reshape((10, 10)),
    columns=[f'col{idx + 1}' for idx in range(10)],
    index=[f'row{idx + 1}' for idx in range(10)],
)

,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10
row1,1,2,3,4,5,6,7,8,9,10
row2,11,12,13,14,15,16,17,18,19,20
row3,21,22,23,24,25,26,27,28,29,30
row4,31,32,33,34,35,36,37,38,39,40
row5,41,42,43,44,45,46,47,48,49,50
row6,51,52,53,54,55,56,57,58,59,60
row7,61,62,63,64,65,66,67,68,69,70
row8,71,72,73,74,75,76,77,78,79,80
row9,81,82,83,84,85,86,87,88,89,90
row10,91,92,93,94,95,96,97,98,99,100


(dataframe-acessing-rows-and-columns)=
## Accessing rows and columns

Now that we know about what makes a `DataFrame` its time to learn about how to access its content. Since there rows and columns, we can can access both individually.

To make things more interesting we will again work with the Iris dataset. Below we load the dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/) using the [`pd.read_csv()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function. This powerful function can load load CSV-like data from a large variety of sources (e.g http(s) or file URLs). For the moment, however, we ask to ignore the details of the function call as we will discuss using this function later in this course.

In [80]:
df_iris = pd.read_csv(
    "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
    names=["sepal length", "sepal width", "petal length", "petal width", "species"],
)

In [81]:
df_iris.sample(5)

,sepal length,sepal width,petal length,petal width,species
104,6.5,3.0,5.8,2.2,Iris-virginica
1,4.9,3.0,1.4,0.2,Iris-setosa
38,4.4,3.0,1.3,0.2,Iris-setosa
62,6.0,2.2,4.0,1.0,Iris-versicolor
90,5.5,2.6,4.4,1.2,Iris-versicolor


Let's first have look at the content of the `DataFrame`. The `.info()` method offers a compact overview thereof.

In [82]:
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal length  150 non-null    float64
 1   sepal width   150 non-null    float64
 2   petal length  150 non-null    float64
 3   petal width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


The columns contain the measured values (`"[sepal/petal_[length|width]"`) in cm units. We note that each value has been stored as a 64-bit floating points value while the `"species"` column, that contains the name of a particular species for which a measurement has been made, is of type `object` (`str` values).

If we now wish to access columns with the Iris specie names we can do this in two different ways. On uses the `[...]` operator while the other uses the [`.loc[...]`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) operator that we already now from our [earlier discussion](series-indexing-with-loc-and-iloc) of `Series`.

In [83]:
df_iris["species"]
# df_iris.sepal_length

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
            ...      
145    Iris-virginica
146    Iris-virginica
147    Iris-virginica
148    Iris-virginica
149    Iris-virginica
Name: species, Length: 150, dtype: object

In [84]:
type(df_iris["species"])

pandas.core.series.Series

In [85]:
df_iris.loc[:, "species"]

0         Iris-setosa
1         Iris-setosa
2         Iris-setosa
3         Iris-setosa
4         Iris-setosa
            ...      
145    Iris-virginica
146    Iris-virginica
147    Iris-virginica
148    Iris-virginica
149    Iris-virginica
Name: species, Length: 150, dtype: object

In [86]:
type(df_iris.loc[:, "species"])

pandas.core.series.Series

Indeed, if we access a single column the result is a `Series` object. You will also have noted that the call to the `.loc[]` method is slightly different to what we are used from the `Series`. The differences is, that now we have the opportunity to index along the rows *and* the columns. The syntax `.loc[:, "species"]` uses a slice `:` to access all rows in the column names "`species`". This is very much like we know it from `ndarray`s like in the following example:

In [87]:
a = np.array([[1, 2], [3, 4], [5, 6]])
a

array([[1, 2],
       [3, 4],
       [5, 6]])

In [88]:
a[:, 0]

array([1, 3, 5])

If we want more than just a single column --- say, all columns related to measurements of the sepal --- we can get their content by passing a `list` of valid column names to the `[...]` operator. Please note that that the order in which the columns are passed does *not* matter. Indeed, we do not need to care too much about the order in which a `DataFrame` contains columns. If we provide a valid name the inner workings of such an object will make sure we get the right content (just like with an associative container as a `dict`).

:::{note} While accessing a *single* column returns a `Series` object, accessing multiple columns returns another `DataFrame` object.
:::

The sketch below summarizes the differences between accessing a single column and multiple columns.

![Accessing a single row vs accessing multiple column](../../_build_img/DataFrameColumnAccess-1.png)

In [99]:
df_iris[["sepal length", "sepal width"]].head(5)

,sepal length,sepal width
0,5.1,3.5
1,4.9,3.0
2,4.7,3.2
3,4.6,3.1
4,5.0,3.6


In [37]:
# Fill the gap!

Again, we can also use the `.loc[]` method to get the same result. The way in which we specify the columns to be accessed is the same as with `[...]`.

In [105]:
(
    df_iris
    .loc[
        :,  # take full slice along axis="rows"
        ["petal length", "petal width"] # select these columns
    ]
    .head()
)

,petal length,petal width
0,1.4,0.2
1,1.4,0.2
2,1.3,0.2
3,1.5,0.2
4,1.4,0.2


If the column names (partially) follow naming schemes that exhibit a certain pattern, the [`.filter()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.filter.html) method is helpful to select the columns according to a given pattern. If we say we want all columns containing the character sequence `"petal"` to can do this:

In [107]:
(
    df_iris
    .filter(like="petal")
    .head()
)

,petal length,petal width
0,1.4,0.2
1,1.4,0.2
2,1.3,0.2
3,1.5,0.2
4,1.4,0.2


We can also use regular expressions with `.filter()` which is particularly interesting if the column names are a bit more compilicated. For instance, in the following example we want to get all columns that end with `"length"` (this is what the `$` symbol means in the context of regular expressions).

In [112]:
(
    df_iris
    .filter(regex="(length)$")
    .head(15)
)

,sepal length,petal length
0,5.1,1.4
1,4.9,1.4
2,4.7,1.3
3,4.6,1.5
4,5.0,1.4
5,5.4,1.7
6,4.6,1.4
7,5.0,1.5
8,4.4,1.4
9,4.9,1.5


Oftentimes, we want to select particular row and column entries at the same time. When selecting particular rows we have to query the index correspondingly as we have dealt with ealier in the section on [`Series`](series-indexing-with-loc-and-iloc). It is important to memorize that there is just a *single* index for all columns in a `DataFrame`. That is, if we query the index we will get information from *all* columns.

Consider the following example in which we query the index label `10` (in `df_iris` the index is just a row counter). The result is a `Series` where the index now consists of the column labels from `df_iris`.

In [111]:
(
    df_iris
    .loc[10]
)

sepal length            5.4
sepal width             3.7
petal length            1.5
petal width             0.2
species         Iris-setosa
Name: 10, dtype: object

Of course we can also use slicing. `DataFrame` also have a `.iloc[]` method available; regarding accessing the row dimension (`axis=:"rows"`) the semantics are the same as for `Series`. Have a look at the following to recall the different behaviour both methods. Note how the output from calling `.iloc[]` is missing the line with the index label `5`!

In [113]:
df_iris.loc[0:5]

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa


In [114]:
df_iris.iloc[0:5]

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


You will not be surprised that the following line is *equivalent* to that using `.loc[]` two cells above. Through `:` we merely express the intent to use all the columns.

In [115]:
df_iris.loc[0:5, :]

,sepal length,sepal width,petal length,petal width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
5,5.4,3.9,1.7,0.4,Iris-setosa


Now we can *combine* row and column access like so:

In [116]:
df_iris.loc[0:5, "sepal length"]

0    5.1
1    4.9
2    4.7
3    4.6
4    5.0
5    5.4
Name: sepal length, dtype: float64

This gives a `Series` because we've only specified a single column name. Of course we can use more than just a single columns like we did before and get another `DataFrame`. The behavior is summarized in the following sketch.

![Accessing rows and columns with `.loc[]`](../../_build_img/DataFrameColumnAccessLoc-1.png)

In [117]:
df_iris.loc[0:5, ["sepal length", "petal length"]]

,sepal length,petal length
0,5.1,1.4
1,4.9,1.4
2,4.7,1.3
3,4.6,1.5
4,5.0,1.4
5,5.4,1.7


It may also get a bit fancier when using boolean masks for the index. For instance, let's say we want the columns `"petal length"` and `"sepal length"` from the `DataFrame` for the species `"Iris-setosa"`. *Note that we have limited the length of the output by chaining the call to `.loc[]` with [`.sample()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) which randomly selects a certain number of rows*. 

In [124]:
# mask = (df_iris["species"] == "Iris-setosa")
(
    df_iris
    .loc[
        df_iris["species"] == "Iris-setosa", # boolean mask
        ["petal length", "sepal length"]     # take only these columns
    ]
    .sample(10)
)

# df_iris["species"] == "Iris-setosa"

,petal length,sepal length
36,1.3,5.5
48,1.5,5.3
11,1.6,4.8
30,1.6,4.8
37,1.5,4.9
7,1.5,5.0
17,1.4,5.1
25,1.6,5.0
40,1.3,5.0
24,1.9,4.8


### Quiz

<span style="display:none" id="5_DataFrames:4">W3sidHlwZSI6ICJtYW55X2Nob2ljZSIsICJhbnN3ZXJfY29scyI6IDEsICJxdWVzdGlvbiI6ICJXaGF0IGRvZXMgdGhlIGV4cHJlc3Npb24gYGRmWydBJ11gIHJldHVybj8gYCdBJ2AgY2FuIGJlIGFzc3VtZWQgdG8gYmUgYSB2YWxpZCBjb2x1bW4gb2YgYGRmYC4iLCAiYW5zd2VycyI6IFt7ImNvcnJlY3QiOiBmYWxzZSwgImFuc3dlciI6ICJBIFB5dGhvbiBsaXN0LiJ9LCB7ImNvcnJlY3QiOiB0cnVlLCAiYW5zd2VyIjogIkEgUGFuZGFzIFNlcmllcyJ9LCB7ImNvcnJlY3QiOiBmYWxzZSwgImFuc3dlciI6ICJBbm90aGVyIE51bVB5IGFycmF5In0sIHsiY29ycmVjdCI6IGZhbHNlLCAiYW5zd2VyIjogIkFub3RoZXIgZGF0YWZyYW1lIn1dfV0=</span>

In [48]:

jupyterquiz.display_quiz("#5_DataFrames:4")

<IPython.core.display.Javascript object>

<span style="display:none" id="5_DataFrames:5">W3sidHlwZSI6ICJtYW55X2Nob2ljZSIsICJhbnN3ZXJfY29scyI6IDEsICJxdWVzdGlvbiI6ICJXaGF0IGlzIHRoZSB0cnVlIGZvciBmb2xsb3dpbmcgcGFpciBvZiBleHByZXNzaW9uczogYGRmWydBJ11gIGFuZCBgZGZbWydBJywgJ0InXV1gPyBgJ0EnYCBhbmQgYCdCJ2AgY2FuIGJvdGggYmUgYXNzdW1lZCB0byBiZSB2YWxpZCBjb2x1bW5zIG9mIGBkZmAuIiwgImFuc3dlcnMiOiBbeyJjb3JyZWN0IjogdHJ1ZSwgImFuc3dlciI6ICJCb3RoIGFyZSB2YWxpZCBleHByZXNzaW9ucyJ9LCB7ImNvcnJlY3QiOiBmYWxzZSwgImFuc3dlciI6ICJUaGUgbGVmdCBpcyBpbnZhbGlkIHdoaWxlIHRoZSByaWdodCBpcyB2YWxpZCJ9LCB7ImNvcnJlY3QiOiBmYWxzZSwgImFuc3dlciI6ICJUaGUgbGVmdCBpcyB2YWxpZCB3aGlsZSB0aGUgcmlnaHQgaXMgaW52YWxpZCJ9LCB7ImNvcnJlY3QiOiB0cnVlLCAiYW5zd2VyIjogIlRoZSBsZWZ0IHJldHVybnMgYSBzZXJpZXMgd2hpbGUgdGhlIHJpZ2h0IHJldHVybnMgYSBkYXRhZnJhbWUifSwgeyJjb3JyZWN0IjogZmFsc2UsICJhbnN3ZXIiOiAiQm90aCByZXR1cm4gZGF0YWZyYW1lcyJ9LCB7ImNvcnJlY3QiOiBmYWxzZSwgImFuc3dlciI6ICJCb3RoIHJldHVybiBzZXJpZXMifV19XQ==</span>

In [49]:

jupyterquiz.display_quiz("#5_DataFrames:5")

<IPython.core.display.Javascript object>

<span style="display:none" id="5_DataFrames:6">W3sidHlwZSI6ICJtYW55X2Nob2ljZSIsICJhbnN3ZXJfY29scyI6IDEsICJxdWVzdGlvbiI6ICJXaGF0IGlzIHRydWUgYWJvdXQgdGhlIGluZGV4PyIsICJhbnN3ZXJzIjogW3siY29ycmVjdCI6IGZhbHNlLCAiYW5zd2VyIjogIlRoZXJlIGlzIG5vIGluZGV4LiJ9LCB7ImNvcnJlY3QiOiB0cnVlLCAiYW5zd2VyIjogIlRoZSBpbmRleCBsYWJlbHMgY2FuIG9ubHkgYmUgYWNjZXNzZWQgd2l0aCB0aGUgYC5sb2NbXWAgbWV0aG9kLiJ9LCB7ImNvcnJlY3QiOiB0cnVlLCAiYW5zd2VyIjogIlRoZXJlIGlzIGEgc2luZ2xlIGluZGV4IGFuZCBpdCBpcyB1c2VkIHRvIGFjY2VzcyB0aGUgcm93IG9mIGVhY2ggY29sdW1uLiJ9LCB7ImNvcnJlY3QiOiBmYWxzZSwgImFuc3dlciI6ICJUaGUgaW5kZXggbGFiZWxzIGNhbiBvbmx5IGJlIGFjY2Vzc2VkIHdpdGggdGhlIGAuaWxvY1tdYCBtZXRob2QuIn0sIHsiY29ycmVjdCI6IGZhbHNlLCAiYW5zd2VyIjogIlRoZXJlIGlzIGEgc2VwYXJhdGUgaW5kZXggZm9yIGVhY2ggY29sdW1uLiJ9XX1d</span>

In [50]:

jupyterquiz.display_quiz("#5_DataFrames:6")

<IPython.core.display.Javascript object>

<span style="display:none" id="5_DataFrames:7">W3sidHlwZSI6ICJtYW55X2Nob2ljZSIsICJhbnN3ZXJfY29scyI6IDEsICJxdWVzdGlvbiI6ICJXaGF0IGEgdmFsaWQgd2F5IHRvIGFjY2VzcyByb3cgYW5kIGNvbHVtbnMgYXQgdGhlIHNhbWUgdGltZT8gTG93ZXIgY2FzZSBsZXR0ZXIgYXJlIGluZGV4IGxhYmVscywgdXBwZXIgY2FzZSBsZXR0ZXJzIGFyZSBjb2x1bW4gbGFiZWxzLiIsICJhbnN3ZXJzIjogW3siY29ycmVjdCI6IHRydWUsICJhbnN3ZXIiOiAiIiwgImNvZGUiOiAiZGYubG9jW1wiYVwiOlwiY1wiLCBbXCJBXCIsIFwiQlwiXV0ifSwgeyJjb3JyZWN0IjogdHJ1ZSwgImFuc3dlciI6ICIiLCAiY29kZSI6ICJkZi5sb2NbXCJhXCI6XCJjXCIsIFtcIkJcIiwgXCJBXCJdXSJ9LCB7ImNvcnJlY3QiOiB0cnVlLCAiYW5zd2VyIjogIiIsICJjb2RlIjogImRmLmxvY1tcImFcIjpcImNcIl1bW1wiQVwiLCBcIkJcIl1dIn0sIHsiY29ycmVjdCI6IGZhbHNlLCAiYW5zd2VyIjogIiIsICJjb2RlIjogImRmW1wiQVwiLCBcIkJcIl0ubG9jW1wiYTpkXCJdIn0sIHsiY29ycmVjdCI6IGZhbHNlLCAiYW5zd2VyIjogIiIsICJjb2RlIjogImRmW1wiQVwiLCBcIkJcIiwgXCJhOmRcIl0ifV19XQ==</span>

In [51]:

jupyterquiz.display_quiz("#5_DataFrames:7")

<IPython.core.display.Javascript object>

Feel free to experiment with following `DataFrame` to answer this question.

In [52]:
df = pd.DataFrame({"A": range(1, 5), "B": range(10, 50, 10)}, index=list("abcd"))
df

,A,B
a,1,10
b,2,20
c,3,30
d,4,40


### Exercises

#### Accessing multiple columns

Access the columns `"sepal length"`, `"petal width"` und `"species"` *at the same time* in two different ways.

First solution.

In [126]:
column_names = ["sepal length", "petal width", "species"]
df_iris[column_names].head()

,sepal length,petal width,species
0,5.1,0.2,Iris-setosa
1,4.9,0.2,Iris-setosa
2,4.7,0.2,Iris-setosa
3,4.6,0.2,Iris-setosa
4,5.0,0.2,Iris-setosa


Second solution.

In [127]:
df_iris.loc[:, column_names].head()

,sepal length,petal width,species
0,5.1,0.2,Iris-setosa
1,4.9,0.2,Iris-setosa
2,4.7,0.2,Iris-setosa
3,4.6,0.2,Iris-setosa
4,5.0,0.2,Iris-setosa


#### Get columns by pattern

Use the `.loc[]` method to select columns that contain the the substring `"width"` from the `df_iris` `DataFrame`. 

:::{note} No, the solution is not to just specify `["sepal width", "petal width"]` manually. Rather think about a programmatic way to generate such a `list`. It actually should be possible to just substitute `"width"` with `"length"` to get `["sepal length", "petal length"]`.
:::

Programmatically generate a `list` containing the relevant column names. Use this list with the `.loc[]` method.

In [131]:
columns_to_select = [
    colname 
    for colname in df_iris.columns
    if "width" in colname
]

df_iris.loc[:, columns_to_select].head()

,sepal width,petal width
0,3.5,0.2
1,3.0,0.2
2,3.2,0.2
3,3.1,0.2
4,3.6,0.2


Programmatically generate a `bool`ean mask that has `True` entries at those positions where `df_iris.columns` has entries containing `"width"`. Use this boolean mask with the `.lop[]` method.

In [133]:
(
    df_iris
    .loc[:, ["width" in colname for colname in df_iris.columns]]
    .head()
)

,sepal width,petal width
0,3.5,0.2
1,3.0,0.2
2,3.2,0.2
3,3.1,0.2
4,3.6,0.2


#### Accessing rows and columns

For which species are the data instances in which the sepal length > 6 cm *and* (at the same time!) petal length > 3.5 cm? How many instances are there per species. Use boolean masks and method chaining.

*Note*: The are multiple cells in which you can write a solution because there are several ways to solve this task.

In [138]:
(
    df_iris
    .loc[df_iris["sepal length"] > 6]
    .loc[df_iris["petal length"] > 3.5]
    ["species"]
    .value_counts()
)

species
Iris-virginica     41
Iris-versicolor    20
Name: count, dtype: int64

In [139]:
(
    df_iris
    .loc[(df_iris["sepal length"] > 6) & (df_iris["petal length"] > 3.5)]
    ["species"]
    .value_counts()
)

species
Iris-virginica     41
Iris-versicolor    20
Name: count, dtype: int64

In [142]:
(
    df_iris
    .query("(`sepal length` > 6) & (`petal length` > 3.5)")
    ["species"]
    .value_counts()
)

species
Iris-virginica     41
Iris-versicolor    20
Name: count, dtype: int64

#### Replacing values

Assume the data for the specie `"Iris-setosa"` is not usable anymore. Change all entries containing *measured values* (e.g. sepal length / width) in rows with `"Iris-setosa"` inside the `DataFrame`s `df_tmp1`, and `df_tmp2` to `np.nan`.

* `df_tmp1`: Use sequential indexing `df_tmp1[cols][rows]`. That is, first access all relevant columns with `[cols]` where `cols` is a placeholder for the corresponding columns. Secondly, access all relevent rows with `[rows]`.
* `df_tmp2`: Use indexing with the `.loc[]` method.

For both approaches check if the changes have come into effect.

In [143]:
df_tmp1 = df_iris.copy(deep=True)
df_tmp2 = df_iris.copy(deep=True)

Sequential indexing for `df_tmp1`.

In [154]:
data_columns = df_tmp1.columns.difference(["species"])
# set(df_iris.columns) - {"species"}
df_tmp1[data_columns][df_tmp1["species"] == "Iris-setosa"] = np.nan

/tmp/ipykernel_15117/2345144162.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp1[data_columns][df_tmp1["species"] == "Iris-setosa"] = np.nan
/tmp/ipykernel_15117/2345144162.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp1[data_columns][df_tmp1["species"] == "Iris-setosa"] = np.nan


Use the `.loc[]` method for `df_tmp2`.

In [157]:
df_tmp2.loc[df_tmp2["species"] == "Iris-setosa", data_columns] = np.nan
df_tmp2

,sepal length,sepal width,petal length,petal width,species
0,NaN,NaN,NaN,NaN,Iris-setosa
1,NaN,NaN,NaN,NaN,Iris-setosa
2,NaN,NaN,NaN,NaN,Iris-setosa
3,NaN,NaN,NaN,NaN,Iris-setosa
4,NaN,NaN,NaN,NaN,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
